# TP22 : Exploration d'un vrai graphe
Le but de ce TP est d'explorer un graphe provenant de données réelles. Il s'agit ici d'explorer les followers à partir du compte [@ENSdeLyon](https://twitter.com/ENSdeLyon). (J'ai emprunté le fichier à un collègue.)

Les données se trouvent dans le fichier `ENSdeLyon.graph`.

Elles ont le format suivant:
* la première ligne contient le nombre de comptes répertoriés, soit $n$;
* les $n$ lignes suivantes contiennent les noms des comptes;
* la ligne suivante contient le nombre de liens de type `followers`, soit $f$;
* les $f$ lignes suivantes contiennent chacune deux entiers : l'indice du compte qui suit et l'indice du compte qui est suivi (les comptes sont numérotés à partir de 0).

### Exercice 1 : Taille
En allant chercher les informations dans le fichier (avec du code), combien y a-t-il de comptes répertoriés dans le fichier `ENSdeLyon.graph`(n'oubliez pas le chemin d'accès complet si vous êtes sur le serveur de TP) ? Combien y a-t-il de liens ?   

Si on considère le graphe orienté dont les sommets sont les comptes et les arcs les relations suiveur/suivi, quelle est la proportion d'arcs dans le graphe par rapport au plus grand nombre d'arcs possibles?

(Laissez le code que vous écrivez pour avoir la réponse dans la cellule suivante, en plus de compléter les valeurs.)

In [2]:
nb_accounts = 0    #à modifier
nb_follows  = 0    #à modifier
proportion  = 0    #à modifier
###BEGIN SOLUTION
f = open('ENSdeLyon.graph')
ns = int(f.readline())
sommets = []
for _ in range(ns):
    sommets.append(f.readline()[:-1])
na = int(f.readline())
adj = [ [] for _ in range(ns) ]
for _ in range(na):
    origine, cible = map(int, f.readline().split(' '))
    adj[origine].append(cible)
f.close()

nb_accounts = len(sommets)
nb_follows  = sum([len(a) for a in adj])
proportion  = nb_follows/nb_accounts**2
###END SOLUTION

In [3]:
###BEGIN HIDDEN TESTS
from math import fabs
assert(nb_accounts == 8418)
assert(nb_follows  == 305288)
assert(fabs(proportion - 0.004308160662861337) < 0.00001)    #à modifier
###END HIDDEN TESTS

### Exercice 2 : Coder le graphe
En principe, la réponse à l'exercice précédent a dû vous convaincre que ce n'est pas une bonne idée de représenter ce graphe avec une matrice d'adjacence. On va donc représenter le graphe avec une liste d'adjacence.

Écrire et documenter une fonction `graphe_twitter` qui prend en argument un nom de fichier au même format que `ENSdeLyon.graph` et renvoie un couple dont le premier argument est la liste des comptes dans leur ordre d'apparition dans le fichier, et le deuxième argument la liste d'adjacence correspondante.

In [4]:
###BEGIN SOLUTION
def graphe_twitter(fichier):
    f = open(fichier)
    ns = int(f.readline())
    sommets = []
    for _ in range(ns):
        sommets.append(f.readline()[:-1])
    na = int(f.readline())
    adj = [ [] for _ in range(ns) ]
    for _ in range(na):
        origine, cible = map(int, f.readline().split(' '))
        adj[origine].append(cible)
    f.close()
    return sommets, adj
###END SOLUTION

In [5]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')
assert(sommets[17] == 'MarioGhossoub')
assert(adj[17] == [5443,1113,5720,8305,821,8370,5658,1327,3680,5875,7258,3783,2956,4812,3541,1214])
assert(sommets[2022] == 'cosyeli')
assert(adj[2022] == [])

### Exercice 3 : Sous-graphe induit
Si $G=(S,A)$ est un graphe et $S'\subseteq S$ un ensemble de sommets de $G$, alors le sous-graphe de $G$ induit par $S'$ est le graphe $G'=(S',A')$ où $$A'=\{(s,t)\in A\mid s\in S'\text{ et }t\in S'\}\:.$$

Écrire et documenter une fonction `sous_graphe_induit` qui prend en argument une liste d'adjacence telle que récupérée par la fonction `graphe_twitter` et un entier $n$ et extrait le sous-graphe induit par les sommets $\{0,1,\dots,n-1\}$.

In [6]:
###BEGIN SOLUTION
def sous_graphe_induit(adj, n):
    return [ [c for c in a if c<n ] for a in adj[:n] ]
###END SOLUTION

In [7]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')
adj2 = sous_graphe_induit(adj, 2500)
assert(adj2[17] == [1113, 821, 1327, 1214])
assert(adj2[2022] == [])

### Exercice 4 : Degrés sortants
1. Écrire et documenter une fonction `degre_sortant_moyen` qui prend en argument une liste d'adjacences comme récupérée de la fonction `graphe_twitter`, et renvoie le degré sortant moyen d'un sommet dans le graphe correspondant.

In [8]:
###BEGIN SOLUTION
def degre_sortant_moyen(adj):
    return sum([ len(a) for a in adj ]) / len(adj)
###END SOLUTION

In [9]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')
degre_sortant_moyen(adj)

36.26609645996674

2. Écrire et documenter une fonction `degre_sortant_max` qui prend en argument une liste de sommets et une liste d'adjacences comme récupérées de la fonction `graphe_twitter`, et renvoie le nom du sommet ayant le degré sortant maximal (si ce sommet n'est pas unique, le nom du premier ayant cette propriété dans la liste des sommets).

In [10]:
###BEGIN SOLUTION
def degre_sortant_max(sommets, adj):
    dmax = -1
    smax = 0
    for s in range(len(sommets)):
        if len(adj[s]) > dmax:
            dmax = len(adj[s])
            smax = s
    return sommets[smax]
###END SOLUTION

In [17]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')
assert(degre_sortant_max(sommets, adj) == 'EditionsScpo')
sommets800 = sommets[:800]
adj800 = sous_graphe_induit(adj, 800)
assert(degre_sortant_max(sommets800, adj800) == 'CbdLes')

### Exercice 5 : Degré entrant moyen
Écrire et documenter une fonction `degre_entrant_moyen` qui prend en argument une liste d'adjacences comme récupérée de la fonction `graphe_twitter`, et renvoie le degré entrant moyen d'un sommet dans le graphe correspondant. Faites en sorte que l'exécution de votre fonction ne soit pas trop longue.

In [10]:
###BEGIN SOLUTION
def degre_entrant_moyen(adj):
    return degre_sortant_moyen(adj)
###END SOLUTION

Tester le temps d'exécution sur un sous-graphe de taille 200, puis de taille 1000 avant de lancer les tests sur des sous-graphes plus grands (chez moi c'est respectivement moins de 100 micro-secondes et moins de 300 micro-secondes, sur le serveur de TP):

In [40]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')

sommets200 = sommets[:200]
adj200 = sous_graphe_induit(adj, 200)

sommets1000 = sommets[:1000]
adj1000 = sous_graphe_induit(adj, 1000)

In [41]:
%%time
from math import fabs

assert(fabs(degre_entrant_moyen(adj200) - 0.23) < 0.00001)

CPU times: user 37 µs, sys: 1e+03 ns, total: 38 µs
Wall time: 41.5 µs


In [48]:
%%time
from math import fabs

assert(fabs(degre_entrant_moyen(adj1000) - 1.275) < 0.00001)

CPU times: user 258 µs, sys: 7 µs, total: 265 µs
Wall time: 272 µs


In [35]:
sommets, adj = graphe_twitter('ENSdeLyon.graph')
###BEGIN HIDDEN TESTS
from math import fabs
assert(fabs(degre_entrant_moyen(adj)) - 36.26609645996674 < 0.00001)
###END HIDDEN TESTS

<div class="alert alert-success">
    <h2>Les points à retenir</h2>
    
* définition du sous-graphe induit par un ensemble de sommets
* la somme des degrés entrants est égale à la somme des degrés sortants
</div>